In [11]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import random
import time
import logging

In [28]:
logging.basicConfig(filename='api_scraper.log', 
                    level=logging.DEBUG)

In [9]:
def calculate_pagecount(total_items):
    if int(total_items)%100 ==0:
        pages = int(total_items)//100
    else:
        pages = (int(total_items)//100)+1
    return pages

## Boardgame Classification

In [35]:
ID = 306882

In [101]:
api_adress = f"https://www.boardgamegeek.com/xmlapi2/thing?id=306882&ratingcomments=1&page=1"

response = requests.get(api_adress)
response.status_code

200

In [102]:
xml = response.text

In [103]:
soup = BeautifulSoup(xml, 'xml')

In [104]:
print(soup.prettify())

<?xml version="1.0" encoding="utf-8"?>
<items termsofuse="https://boardgamegeek.com/xmlapi/termsofuse">
 <item id="306882" type="boardgame">
  <thumbnail>
   https://cf.geekdo-images.com/yE1xMCPzckKh9V9CStNFEw__thumb/img/zFQhF83FC3HYETEMz3R8_R0B2mQ=/fit-in/200x150/filters:strip_icc()/pic5350626.jpg
  </thumbnail>
  <image>
   https://cf.geekdo-images.com/yE1xMCPzckKh9V9CStNFEw__original/img/cpsGDBjzoKWhE8u5SojuSYafNmg=/0x0/filters:format(jpeg)/pic5350626.jpg
  </image>
  <name sortindex="1" type="primary" value="Railroad Ink Challenge: Shining Yellow Edition"/>
  <name sortindex="1" type="alternate" value="Railroad Ink Challenge: Edition Sonnengelb"/>
  <name sortindex="1" type="alternate" value="Railroad Ink Challenge: Goudgele Versie"/>
  <description>
   Railroad Ink Challenge is a quick-playing roll-and-write game for 1 to 4 players. Grab a board and a dry-erase marker, and get ready to reach networking nirvana! Roll the dice and draw the routes to connect the exits around your boa

In [72]:
soup.find_all('comments', attrs={page:'2'})

[]

In [ ]:
categories = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamecategory'})]
categories

In [ ]:
mechanics = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamemechanic'})]
mechanics

In [ ]:
family = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamefamily'})]
family

In [ ]:
expansions = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgameexpansion'})]
expansions

In [ ]:
integrations = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgameintegration'})]
integrations

In [ ]:
designers = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamedesigner'})]
designers

In [ ]:
publishers = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamepublisher'})]
publishers 

## User Ratings

In [ ]:
api_adress = "https://www.boardgamegeek.com/xmlapi2/thing?id=306882&ratingcomments=1&page=1"

response = requests.get(api_adress)
response.status_code

In [ ]:
xml = response.text
soup = BeautifulSoup(xml, 'xml')

In [ ]:
#print(soup.prettify())

In [ ]:
page = soup.find('comments').get('page')
page

In [ ]:
total_items = soup.find('comments').get('totalitems')
total_items

In [ ]:
pagecount = calculate_pagecount(total_items)
pagecount

In [ ]:
ratings = []
users = []
comments = []
for comment in soup.find_all('comment'):
    ratings.append(comment.get('rating'))
    users.append(comment.get('username'))
    comments.append(comment.get('value'))
    

In [ ]:
boardgame_id = [ID] * len(ratings)

In [ ]:
ratings = pd.DataFrame({
    'id':boardgame_id,
    'user':users,
    'rating':ratings,
    'comment':comments
})

In [ ]:
ratings['rating'].isna().sum()

To DO:
- Loop through pages
- ignore users without ratings
- User IDs?
- include logging
- append csv not dataframe
- check request status

## Loop over rating pages

In [29]:
boardgames =pd.read_csv('../data/boardgames.csv', index_col='id')
boardgames['categories'] = None
boardgames['mechanics'] = None
boardgames['family'] = None
boardgames['expansions'] = None
boardgames['integrations'] = None
boardgames['designers'] = None
boardgames['publishers'] = None

sleep_default = 2

In [34]:
for ID in [4974, 6927, 8139, 8993, 13121]:
    logging.debug(f'movie id: {ID}')
    
    user_ratings = []
    users = []
    comments = []
    ratings = pd.DataFrame()
    page = 1
    
    # request page 1 of game overview with user ratings and comments
    api_adress = f"https://www.boardgamegeek.com/xmlapi2/thing?id={ID}&ratingcomments=1&page={page}"
    response = requests.get(api_adress)  
    logging.info(f'status {response.status_code} for {api_adress}')    
    
    xml = response.text
    soup = BeautifulSoup(xml, 'xml')
    
    # get the number of pages of ratings
    total_items = soup.find('comments').get('totalitems')
    logging.debug(f'total items: {total_items}')
    pagecount = calculate_pagecount(total_items)
    logging.debug(f'pagecount: {pagecount}')
    logging.debug(f'start scraping page: {page}')

    
    # extend boardgames dataframe 
    categories = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamecategory'})]
    boardgames.loc[ID,'categories'] = ', '.join(categories)
    mechanics = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamemechanic'})]
    boardgames.loc[ID,'mechanics'] = ', '.join(mechanics)
    family = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamefamily'})]
    boardgames.loc[ID,'family'] = ', '.join(family)
    expansions = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgameexpansion'})]
    boardgames.loc[ID,'expansions'] = ', '.join(expansions)
    integrations = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgameintegration'})]
    boardgames.loc[ID,'integrations'] = ', '.join(integrations)
    designers = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamedesigner'})]
    boardgames.loc[ID,'designers'] = ', '.join(designers)
    publishers = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamepublisher'})]
    boardgames.loc[ID,'publishers'] = ', '.join(publishers)
    
    # update number of voters
    boardgames.loc[ID, 'num_voters'] = total_items
    
    boardgames.to_csv('../data/boardgames_extend.csv', index=False)
    
    #get user ratings from page 1
    for comment in soup.find_all('comment'):
        user_ratings.append(comment.get('rating'))
        users.append(comment.get('username'))
        comments.append(comment.get('value')) 
        
    boardgame_id = [ID] * len(user_ratings)

    ratings = pd.DataFrame({
        'id':boardgame_id,
        'user':users,
        'rating':user_ratings,
        'comment':comments
    })
        
    ratings.to_csv('../data/ratings.csv', index=False, header=None, mode='a')
    
    # if more than one page of comments send request for each page
    if pagecount > page:
        page = page + 1

        while page <= pagecount:
            user_ratings = []
            users = []
            comments = []
            ratings = pd.DataFrame()
            
            api_adress = f"https://www.boardgamegeek.com/xmlapi2/thing?id={ID}&ratingcomments=1&page={page}"
            response = requests.get(api_adress)
            logging.info(f'status {response.status_code} for {api_adress}')
            
            if response.status_code != 200:
                logging.warning(f'STATUS {response.status_code}, increase sleep time and try again')
                time.sleep(10)
                sleep_default = 5
                
            else:
            
                xml = response.text
                soup = BeautifulSoup(xml, 'xml')

                for comment in soup.find_all('comment'):
                    user_ratings.append(comment.get('rating'))
                    users.append(comment.get('username'))
                    comments.append(comment.get('value'))

                boardgame_id = [ID] * len(user_ratings)

                ratings = pd.DataFrame({
                    'id':boardgame_id,
                    'user':users,
                    'rating':user_ratings,
                    'comment':comments
                })

                ratings.to_csv('../data/ratings.csv', index=False, header=False, mode='a')

                page = page + 1
                time.sleep(sleep_default+random.random())


In [107]:
boardgames.loc[822:].index

Int64Index([   822,     13,  68448,  36218,   9209, 178900, 167791,  31260,
              3076, 173346,
            ...
            137184, 137355, 137427, 138223, 138518, 138578, 138730, 139581,
            139637, 139930],
           dtype='int64', name='id', length=44999)

In [108]:
boardgames.loc[70323]

rank                                    320
name                          King of Tokyo
links           /boardgame/70323/king-tokyo
num_voters                            58758
categories                             None
mechanics                              None
family                                 None
expansions                             None
integrations                           None
designers                              None
publishers                             None
Name: 70323, dtype: object

In [124]:
boardgames['num_voters'] = boardgames['num_voters'].astype('int')

In [141]:
boardgames[boardgames['num_voters']<27000].head(20)

,rank,name,links,num_voters,categories,mechanics,family,expansions,integrations,designers,publishers
id,,,,,,,,,,,
3955,1294.0,BANG!,/boardgame/3955/bang,26935,None,None,None,None,None,None,None
72125,54.0,Eclipse,/boardgame/72125/eclipse,26445,None,None,None,None,None,None,None
205059,34.0,Mansions of Madness: Second Edition,/boardgame/205059/mansions-madness-second-edition,26399,None,None,None,None,None,None,None
9220,1114.0,Saboteur,/boardgame/9220/saboteur,26380,None,None,None,None,None,None,None
136063,394.0,Forbidden Desert,/boardgame/136063/forbidden-desert,26217,None,None,None,None,None,None,None
10630,145.0,Memoir '44,/boardgame/10630/memoir-44,26116,None,None,None,None,None,None,None
148949,128.0,Istanbul,/boardgame/148949/istanbul,25957,None,None,None,None,None,None,None
15062,419.0,Shadows over Camelot,/boardgame/15062/shadows-over-camelot,25677,None,None,None,None,None,None,None
42,95.0,Tigris & Euphrates,/boardgame/42/tigris-euphrates,25647,None,None,None,None,None,None,None


In [139]:
boardgames.loc[:15062]['num_voters'].sum()

3770584

In [138]:
boardgames.loc[:15062]['num_voters'].count()

90

In [128]:
ratings_loaded = pd.read_csv('../data/ratings.csv', names=['id', 'user', 'rating', 'comment'])

In [130]:
ratings_loaded.shape

(2532972, 4)

In [131]:
ratings_loaded.value_counts('user')

user
squaredeh          90
tugurio77          82
CuJ1976            74
Doel               69
RonCarlson01       68
                   ..
GrumpyMonkey246     1
eltrex              1
Grumphle            1
Grumpae             1
chitons             1
Length: 278116, dtype: int64

In [121]:
ratings_loaded.value_counts('id')

id
30549     105666
822       105191
13        104739
68448      86952
36218      79957
9209       73673
178900     70783
167791     69052
31260      64920
3076       64432
173346     64131
40692      63074
148228     61399
169786     60921
14996      60663
70323      59106
129622     58637
230802     57573
2651       57495
39856      52392
478        49343
163412     49000
266192     48838
84876      48398
28143       6300
dtype: int64

In [122]:
ratings_loaded[ratings_loaded['id']==266192]

,id,user,rating,comment
1567497,266192,ogzz,10.0,This is my favorite game outside of the Pandem...
1567498,266192,Jamiekysn,10.0,NaN
1567499,266192,Stumpeone,10.0,I'm probably going to buy everything that gets...
1567500,266192,garbagerunner,10.0,NaN
1567501,266192,Kullholen,10.0,NaN
...,...,...,...,...
1616330,266192,LuzhinLives,1.0,NaN
1616331,266192,Basowillow,1.0,Stonemaier games published another unbalanced ...
1616332,266192,xiaoxiaocat,1.0,NaN
1616333,266192,akneeland,1.0,NaN
